In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

from src.configuration.constants import PROCESSED_DATA_DIRECTORY, ROOT_DIRECTORY, INTERIM_DATA_DIRECTORY, LOGS_DATA_DIRECTORY
from src.models.layers import *
from src.utils.dataset import load_dataset


import collections
import logging
import os
import pathlib
import re
import string
import sys
import time
import math
import tempfile
from functools import partial

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import pandas as pd

from orion.data import load_signal
from orion import Orion
from orion.data import load_anomalies

import tensorflow as tf
from functools import partial
from mlprimitives import load_primitive

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from orion.evaluation.contextual import contextual_f1_score
import pandas as pd
import random


from mlprimitives import load_primitive

logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

# Transformer Experiments

## Positional Encoding

In [2]:
n, d = 100, 55
pos_encoding = positional_encoding(n, d)

print(pos_encoding.shape)
pos_encoding = pos_encoding[0]

(1, 100, 55)


## Masking

In [3]:
# 0 are pads
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

<tf.Tensor 'strided_slice_1:0' shape=(3, 1, 1, 5) dtype=float32>

In [4]:
# look-ahead mask
# x = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(x.shape[1])
temp

<tf.Tensor 'sub:0' shape=(5, 5) dtype=float32>

## Multihead Attention

In [5]:
temp_mha = MultiHeadAttention(d_model=d, num_heads=5)
y = tf.random.uniform((1, n, d))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

(TensorShape([Dimension(1), Dimension(100), Dimension(55)]),
 TensorShape([Dimension(1), Dimension(5), Dimension(100), Dimension(100)]))

## Point wise feed forward network

In [6]:
x = tf.random.uniform((64, n, d), minval=-1, maxval=1)
sample_ffn = point_wise_feed_forward_network(d, d*2)
sample_ffn(x).shape

TensorShape([Dimension(64), Dimension(100), Dimension(55)])

## Encoder

In [7]:
sample_encoder_layer = EncoderLayer(d, 5, d*4)

sample_encoder_layer_output = sample_encoder_layer(x, False, None)

sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

TensorShape([Dimension(64), Dimension(100), Dimension(55)])

In [8]:
sample_encoder = Encoder(num_layers=2, d_model=d, num_heads=5,
                         dff=d*2, input_vocab_size=None,
                         maximum_position_encoding=10000)

sample_encoder_output = sample_encoder(x, training=False, mask=None)

sample_encoder_output.shape  # (batch_size, input_seq_len, d_model)

TensorShape([Dimension(64), Dimension(100), Dimension(55)])

## Decoder

In [9]:
sample_decoder_layer = DecoderLayer(d, 5, d*4)

sample_decoder_layer_output, _, _ = sample_decoder_layer(x, sample_encoder_layer_output, False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)

TensorShape([Dimension(64), Dimension(100), Dimension(55)])

In [10]:
sample_decoder = Decoder(num_layers=2, d_model=d, num_heads=5,
                         dff=d*4, target_vocab_size=8000,
                         maximum_position_encoding=5000)

output, attn = sample_decoder(x,
                              enc_output=sample_encoder_output,
                              training=False,
                              look_ahead_mask=None,
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

(TensorShape([Dimension(64), Dimension(100), Dimension(55)]),
 TensorShape([Dimension(64), Dimension(5), Dimension(100), Dimension(100)]))

## Transformer

In [11]:
sample_transformer = Transformer(
    num_layers=2, d_model=d, num_heads=5, dff=d*4,
    input_vocab_size=8500, target_vocab_size=1,
    pe_input=10000, pe_target=6000)

fn_out = sample_transformer(x, training=False)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

TensorShape([Dimension(64), Dimension(100), Dimension(1)])

# Attention TadGAN

## Dataset

### Loading

In [12]:
dataset = load_dataset('interim', 'MSL')

train_split = dataset['train']
test_split = dataset['test']
anomalies_split = dataset['anomaly']

signal_to_dataset = {}
for signal in train_split.signal.unique():
    signal_to_dataset[signal] = {
        'train': train_split[train_split.signal == signal],
        'test': test_split[test_split.signal == signal],
        'anomaly': anomalies_split[anomalies_split.signal == signal],
    }

signal_to_dataset.keys()

dict_keys(['M-1', 'M-2', 'M-3', 'M-7', 'T-5', 'T-4', 'M-6', 'M-4', 'M-5', 'F-8', 'S-2', 'D-15', 'P-14', 'P-15', 'D-14', 'D-16', 'F-7', 'P-11', 'F-5', 'F-4', 'P-10', 'T-13', 'T-12', 'T-9', 'T-8', 'C-1', 'C-2'])

In [13]:
m2_dataset = signal_to_dataset['M-2']

train = m2_dataset['train']
test = m2_dataset['test']
anomalies = m2_dataset['anomaly']

index_train = train['index'].astype(int)
X_train = train[['anomaly'] + list(train.columns)[5:]]

index_test = test['index'].astype(int).reset_index(drop=True)
X_test = test[['anomaly'] + list(test.columns)[5:]]

In [14]:
train

,source,name,signal,anomaly,index,0,1,2,3,4,...,45,46,47,48,49,50,51,52,53,54
0,NASA,MSL,M-2,0,0,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NASA,MSL,M-2,0,1,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NASA,MSL,M-2,0,2,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NASA,MSL,M-2,0,3,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NASA,MSL,M-2,0,4,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,NASA,MSL,M-2,0,5,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NASA,MSL,M-2,0,6,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NASA,MSL,M-2,0,7,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,NASA,MSL,M-2,0,8,-0.748738,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,NASA,MSL,M-2,0,9,-1.093982,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Pre-Processing

In [15]:
def fit_processing(X, index):
    primitives = []
    
    
    # This primitive is an imputation transformer for filling missing values
    params = {
        'X': X
    }
    primitive = load_primitive('sklearn.impute.SimpleImputer', arguments=params)
    primitive.fit()
    primitives.append(primitive)
    X = primitive.produce(X=X)
    print(primitive, X.shape)
    
    # This primitive transforms features by scaling each feature to a given range
    params = {
        "feature_range": [-1, 1], 
        'X': X,
    }
    primitive = load_primitive('sklearn.preprocessing.MinMaxScaler', arguments=params)
    primitive.fit()
    primitives.append(primitive)
    X = primitive.produce(X=X)
    print(primitive, X.shape)
    
    # Uses a rolling window approach to create the sub-sequences out of time series data
    params = {
        "target_column": 0, 
        "window_size": 100, 
        'target_size': 1, 
        'step_size': 1
    }
    primitive = load_primitive('mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences',
                               arguments=params)
    primitives.append(primitive)
    X, y, index, target_index = primitive.produce(X=X, index=index)

    # Target / target size is the next interval that is trying to predict.
    # Index is the start of the interval
    print(primitive, X.shape, y.shape, index.shape, target_index.shape)
    
    return X, index, primitives 

In [16]:
X, index, primitives = fit_processing(X_train, index_train)

MLBlock - sklearn.impute.SimpleImputer (2208, 56)
MLBlock - sklearn.preprocessing.MinMaxScaler (2208, 56)
MLBlock - mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences (2108, 100, 56) (2108, 1) (2108,) (2108,)


In [17]:
X_train = X[:, :, 1:]
y_train = np.expand_dims(X_train[:, :, 0], 2)
# y_train = X[:, :, 1:]
index_train = index

X_train.shape, y_train.shape, index_train.shape

((2108, 100, 55), (2108, 100, 1), (2108,))

In [18]:
def produce_processing(X, index, primitives):
    X = primitives[0].produce(X=X)
    X = primitives[1].produce(X=X)
    X, y, index, target_index = primitives[2].produce(X=X, index=index)
    return X, index

In [19]:
X, index = produce_processing(X_test, index_test, primitives)

In [20]:
X_test = X[:, :, 1:]
y_test = np.expand_dims(X_test[:, :, 0], 2)
# y_test = X[:, :, 1:]
index_test = index
labels_test = np.array([1 if sum(i) > 0 else 0 for i in X[:, :, 0]])


X_test.shape, y_test.shape, index_test.shape, labels_test.shape

((2177, 100, 55), (2177, 100, 1), (2177,), (2177,))

## Model

In [55]:
X_train[0].shape

(100, 55)

In [21]:
input_shape = X_train[0].shape
target_shape = y_train[0].shape
latent_dim = 20
latent_shape = (latent_dim, 1)

shape = np.asarray(X_train)[0].shape
length = shape[0]
target_shape = np.asarray(y_train)[0].shape

generator_reshape_dim = (length // 2) * target_shape[-1]
generator_reshape_shape = (length // 2, target_shape[-1])
encoder_reshape_shape = latent_shape

encoder_input_shape = shape
generator_input_shape = latent_shape
critic_x_input_shape = target_shape
critic_z_input_shape = latent_shape

lstm_units = 100
dense_units = 20

learning_rate=0.0005
epochs=70
batch_size=64
iterations_critic=5

In [24]:
def build_encoder_layer(input_shape, lstm_units, dense_units, encoder_reshape_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=lstm_units, return_sequences=True)))
    # model.add(Encoder(num_layers=2, d_model=input_shape[-1], num_heads=5,
    #                   dff=input_shape[-1]*4, input_vocab_size=None,
    #                   maximum_position_encoding=10000))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.Reshape(target_shape=encoder_reshape_shape))
    return tf.keras.Model(x, model(x))
              
encoder = build_encoder(
    input_shape=encoder_input_shape,
    lstm_units=lstm_units,
    dense_units=dense_units,
    encoder_reshape_shape=encoder_reshape_shape,
)
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 100, 55)]         0         
_________________________________________________________________
sequential_13 (Sequential)   (None, 20, 1)             524820    
Total params: 524,820
Trainable params: 524,820
Non-trainable params: 0
_________________________________________________________________


In [25]:
def generator_layer(input_shape, generator_reshape_dim, generator_reshape_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=generator_reshape_dim))
    model.add(tf.keras.layers.Reshape(target_shape=generator_reshape_shape))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    # model.add(Encoder(num_layers=2, d_model=generator_reshape_shape[-1], num_heads=1,
    #                   dff=generator_reshape_shape[-1]*4, input_vocab_size=None,
    #                   maximum_position_encoding=10000))
    
    model.add(tf.keras.layers.UpSampling1D(size=2))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    # model.add(Encoder(num_layers=2, d_model=generator_reshape_shape[-1], num_heads=1,
    #                   dff=generator_reshape_shape[-1]*4, input_vocab_size=None,
    #                   maximum_position_encoding=10000))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=1)))
    # model.add(tf.keras.layers.Dense(units=1))
    model.add(tf.keras.layers.Activation(activation='tanh'))
    return tf.keras.Model(x, model(x))
              
generator = build_generator(
    input_shape=generator_input_shape,
    generator_reshape_dim=generator_reshape_dim,
    generator_reshape_shape=generator_reshape_shape,
)
generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
sequential_14 (Sequential)   (None, 100, 1)            133787    
Total params: 133,787
Trainable params: 133,787
Non-trainable params: 0
_________________________________________________________________


In [26]:
def build_critic_x(input_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1))
    return tf.keras.Model(x, model(x))
              
critic_x = build_critic_x(
    input_shape=critic_x_input_shape
)
critic_x.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 100, 1)]          0         
_________________________________________________________________
sequential_15 (Sequential)   (None, 1)                 67393     
Total params: 67,393
Trainable params: 67,393
Non-trainable params: 0
_________________________________________________________________


In [27]:
def build_critic_z(input_shape, dense_units=20):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=1))
    return tf.keras.Model(x, model(x))
              
critic_z = build_critic_z(
    input_shape=critic_z_input_shape,
)
critic_z.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
sequential_16 (Sequential)   (None, 1)                 861       
Total params: 861
Trainable params: 861
Non-trainable params: 0
_________________________________________________________________


In [28]:
def _wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def _gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)
    return K.mean(gradient_penalty)

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [30]:
generator.trainable = False
encoder.trainable = False

x = tf.keras.Input(shape=input_shape)
y = tf.keras.Input(shape=target_shape)
z = tf.keras.Input(shape=(latent_dim, 1))

x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x(x_) # Fake
valid_x = critic_x(y) # Truth

In [31]:
alpha = K.random_uniform((64, 1, 1))
interpolated_x = (alpha * [y, x_][0]) + ((1 - alpha) * [y, x_][1])
validity_interpolated_x = critic_x(interpolated_x)
partial_gp_loss_x = partial(_gradient_penalty_loss, averaged_samples=interpolated_x)
partial_gp_loss_x.__name__ = 'gradient_penalty'
critic_x_model = tf.keras.Model(inputs=[y, z], outputs=[valid_x, fake_x, validity_interpolated_x])
critic_x_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, partial_gp_loss_x], 
                       optimizer=optimizer, loss_weights=[1, 1, 10])

2022-01-28 14:46:52.426432: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-01-28 14:46:52.440221: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f94cbb68520 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-01-28 14:46:52.440236: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [32]:
fake_z = critic_z(z_)
valid_z = critic_z(z)
alpha = K.random_uniform((64, 1, 1))
interpolated_z = (alpha * [z, z_][0]) + ((1 - alpha) * [z, z_][1])
validity_interpolated_z = critic_z(interpolated_z)
partial_gp_loss_z = partial(_gradient_penalty_loss, averaged_samples=interpolated_z)
partial_gp_loss_z.__name__ = 'gradient_penalty'
critic_z_model = tf.keras.Model(inputs=[x, z], outputs=[valid_z, fake_z,validity_interpolated_z])
critic_z_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,
                                  partial_gp_loss_z], optimizer=optimizer,
                            loss_weights=[1, 1, 10])

In [33]:
critic_x.trainable = False
critic_z.trainable = False
generator.trainable = True
encoder.trainable = True

z_gen = tf.keras.Input(shape=(latent_dim, 1))
x_gen_ = generator(z_gen)
x_gen = tf.keras.Input(shape=input_shape)
z_gen_ = encoder(x_gen)
x_gen_rec = generator(z_gen_)
fake_gen_x = critic_x(x_gen_)
fake_gen_z = critic_z(z_gen_)

encoder_generator_model = tf.keras.Model([x_gen, z_gen], [fake_gen_x, fake_gen_z, x_gen_rec])
encoder_generator_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, 'mse'], 
                                optimizer=optimizer,
                                loss_weights=[1, 1, 10])

### Training

https://gist.github.com/erenon/91f526302cd8e9d21b73f24c0f9c4bb8

In [34]:
fake = np.ones((batch_size, 1))
valid = -np.ones((batch_size, 1))
delta = np.ones((batch_size, 1))

indices = np.arange(X_train.shape[0])

epoch_loss = []

for epoch in range(1, epochs + 1):
    np.random.shuffle(indices)
    X_ = X_train[indices]
    y_ = y_train[indices]

    epoch_g_loss = []
    epoch_cx_loss = []
    epoch_cz_loss = []

    minibatches_size = batch_size * iterations_critic
    num_minibatches = int(X_.shape[0] // minibatches_size)

    for i in range(num_minibatches):
        minibatch = X_[i * minibatches_size: (i + 1) * minibatches_size]
        y_minibatch = y_[i * minibatches_size: (i + 1) * minibatches_size]

        for j in range(iterations_critic):
            x = minibatch[j * batch_size: (j + 1) * batch_size]
            y = y_minibatch[j * batch_size: (j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, 1))
            epoch_cx_loss.append(
                critic_x_model.train_on_batch([y, z], [valid, fake, delta]))
            epoch_cz_loss.append(
                critic_z_model.train_on_batch([x, z], [valid, fake, delta]))
        
        epoch_g_loss.append(
            encoder_generator_model.train_on_batch([x, z], [valid, valid, y]))
        
    cx_loss = np.mean(np.array(epoch_cx_loss), axis=0)
    cz_loss = np.mean(np.array(epoch_cz_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    
    epoch_loss.append([cx_loss, cz_loss, g_loss])
    
    print('Epoch: {}/{}, [Dx loss: {}] [Dz loss: {}] [G loss: {}]'.format(
        epoch, epochs, cx_loss, cz_loss, g_loss))
    

Epoch: 1/70, [Dx loss: [-1.1452382  -4.524018    0.14604482  0.3232734 ]] [Dz loss: [14.6126995  -0.3771688  14.224738    0.07651301]] [G loss: [ -8.444636    -0.13303666 -12.712794     0.44011936]]
Epoch: 2/70, [Dx loss: [-5.3779454 -8.523363   2.2020662  0.094335 ]] [Dz loss: [43.581      -0.12714562 42.618523    0.10896247]] [G loss: [-34.125214    -2.2819173  -33.303905     0.14606084]]
Epoch: 3/70, [Dx loss: [-1.1830037  -7.7433934   6.36148     0.01989108]] [Dz loss: [-6.2242687e+01 -1.5483927e-02 -6.4134575e+01  1.9073802e-01]] [G loss: [76.320145   -6.0929694  80.940186    0.14729302]]
Epoch: 4/70, [Dx loss: [-0.82828426  2.0606198  -3.3637338   0.04748296]] [Dz loss: [-111.70037      -0.25529483 -112.68816       0.12430551]] [G loss: [134.91982      5.475105   127.76443      0.16803025]]
Epoch: 5/70, [Dx loss: [-3.1545906   4.807382   -8.329565    0.03675915]] [Dz loss: [-67.90073     -0.7859203  -68.29033      0.11755184]] [G loss: [89.912994    7.9709587  80.651474    0.1290

In [ ]:
[Dx loss: [-8.7269503e-01 -1.5812571e+01  1.4787490e+01  1.5238333e-02]] 
[Dz loss: [-1.1258742   1.3251286  -3.821981    0.13709784]] 
[G loss: [-10.728219   -15.398585     3.8551104    0.08152536]]

In [33]:
[Dx loss: [-1.7266345  -2.4032445   0.49706748  0.01795427]] 
[Dz loss: [8.509448   3.961609   1.9949051  0.25529343]] 
[G loss: [-1.5865823 -0.4866698 -1.9377236  0.0837811]]
                                                                                                                       

SyntaxError: invalid syntax (1777920837.py, line 1)

## Evaluation

In [35]:
z_ = encoder.predict(X_test)
y_hat = generator.predict(z_)
critic = critic_x.predict(y_test)

In [36]:
# computes an array of anomaly scores based on a combination of reconstruction error and critic output
params = {"rec_error_type": "dtw", "comb": "mult"}

primitive = load_primitive("orion.primitives.tadgan.score_anomalies", 
                           arguments=params)
errors, true_index, true, predictions = primitive.produce(y=y_test, y_hat=y_hat, critic=critic, index=index_test)

errors.shape, true_index.shape

Using TensorFlow backend.


((2276,), (2177,))

In [37]:
errors[1110:2250]

array([2.26213938, 2.25722421, 2.25230904, ..., 3.12313495, 3.33712498,
       3.55063581])

In [38]:
# extracts anomalies from sequences of errors following the approach
params = {
    "window_size_portion": 0.33, 
    "window_step_size_portion": 0.1,
    "fixed_threshold": True
}

primitive = load_primitive("orion.primitives.timeseries_anomalies.find_anomalies", 
                           arguments=params)
e = primitive.produce(errors=errors, index=true_index)

e.shape

(1, 3)

In [39]:
e

array([[1.14900000e+03, 1.20700000e+03, 4.46854429e-01]])

In [42]:
predicted_anomalies = [(int(i[0]), int(i[1])) for i in e]
print(anomalies)
print(predicted_anomalies)
start, end = index_test[0], index_test[-1]
contextual_f1_score(anomalies, predicted_anomalies, start=start, end=end, weighted=True)

  source  name signal  start   end
0   NASA  SMAP    M-2   1110  2250
[(1149, 1207)]


0.09833333333333333

In [41]:
1153, 1207

(1153, 1207)

In [39]:
0.09197324414715718
0.09674728940783987

0.09674728940783987